In [33]:
!pip install boto3
import boto3

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
You should consider upgrading via the 'pip install --upgrade pip' command.


In [34]:
s3 = boto3.resource('s3', 
	 aws_access_key_id = '',
	 aws_secret_access_key = '')

In [45]:
try:s3.create_bucket(Bucket='datacont-datz', CreateBucketConfiguration={
    'LocationConstraint': 'us-west-2'})
except Exception as e:
    print (e)

In [46]:
bucket =s3.Bucket("datacont-datz")

In [47]:
bucket.Acl().put(ACL='public-read')

{'ResponseMetadata': {'RequestId': '7ZXQDAQB15HYV4V6',
  'HostId': 'hVaAPB+1hFCPWkLP8OABBegKg1FhIbW00pokHBI1obRRhc5aO7NQew1In6VkmyUIC+rtzKD0SyA=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'hVaAPB+1hFCPWkLP8OABBegKg1FhIbW00pokHBI1obRRhc5aO7NQew1In6VkmyUIC+rtzKD0SyA=',
   'x-amz-request-id': '7ZXQDAQB15HYV4V6',
   'date': 'Tue, 09 Mar 2021 22:09:27 GMT',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 1}}

In [48]:
dyndb =boto3.resource('dynamodb',
                      region_name='us-east-2',
                      aws_access_key_id='',
                      aws_secret_access_key='')

In [49]:
try:
    table =dyndb.create_table(
        TableName='DataTable',
        KeySchema=[
            {
                'AttributeName': 'PartitionKey',
                'KeyType': 'HASH'
            },
            {
                'AttributeName': 'RowKey',
                'KeyType': 'RANGE'
            }
        ],
        AttributeDefinitions=[
            {
                'AttributeName': 'PartitionKey',
                'AttributeType': 'S'
            },
            {
                'AttributeName': 'RowKey',
                'AttributeType': 'S'
            },
        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 5,
            'WriteCapacityUnits': 5
        }
)
except:
    #if there is an exception, the table may already exist.   if so...
    table =dyndb.Table("DataTable")

In [50]:
table.meta.client.get_waiter('table_exists').wait(TableName='DataTable')

In [51]:
print(table.item_count)

0


In [52]:
import csv

In [56]:
with open('/Users/michaeldatz/Desktop/2021-2022/cloudcomputing/hw3/experiments.csv', "rt", encoding="utf8") as csvfile: 
    csvf = csv.reader(csvfile, delimiter=',', quotechar='|')
    next(csvf, None)
    for item in csvf:
        print(item)
        body = open('/Users/michaeldatz/Desktop/2021-2022/cloudcomputing/hw3/'+item[4], 'rb') 
        s3.Object('datacont-datz', item[4]).put(Body=body)
        md = s3.Object('datacont-datz', item[4]).Acl().put(ACL='public-read')
        url = "https://s3-us-east-2.amazonaws.com/datacont-datz/"+item[3] 
        metadata_item = {'PartitionKey': item[0], 'RowKey': item[1],
                         'description' : item[4], 'date' : item[2], 'url':url}
        try: 
            table.put_item(Item=metadata_item)
        except:
            print("item may already be there or another failure")

['experiment1', 'data1', '2/13/21', 'Words..', 'exp1.csv']
['experiment2', 'data2', '2/20/21', 'second experiment', 'exp2.csv']


In [60]:
response = table.get_item( 
	Key={
		'PartitionKey': 'experiment2',
		'RowKey': 'data2' 
	}
)
item = response['Item'] 
print(item)

{'PartitionKey': 'experiment2', 'RowKey': 'data2', 'date': '2/20/21', 'description': 'exp2.csv', 'url': 'https://s3-us-east-2.amazonaws.com/datacont-datz/second experiment'}


In [61]:
response

{'Item': {'PartitionKey': 'experiment2',
  'RowKey': 'data2',
  'date': '2/20/21',
  'description': 'exp2.csv',
  'url': 'https://s3-us-east-2.amazonaws.com/datacont-datz/second experiment'},
 'ResponseMetadata': {'RequestId': 'FF74S370ROSH2G0DMBJOQKIP3JVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Tue, 09 Mar 2021 22:20:41 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '203',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'FF74S370ROSH2G0DMBJOQKIP3JVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '4059314535'},
  'RetryAttempts': 0}}